# SOE-MIA-V1E4-002
## Modulo 2 - Data, open data y Big Data
## Práctiva Nro. 2
### Alumno:

Práctica de Google Colab y PySpark sobre Indicadores de enfermedad cardiaca que se cuentra en el DataSet "Heart Disease Prediction" del repositorio [Kaggle](https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease).  

## 1. Configuración del entorno:

In [1]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy


In [2]:
! python


Python 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> print('hola mundo')
hola mundo
>>> qui()
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'qui' is not defined. Did you mean: 'quit'?
>>> quit()


### Instalación de PySpark

In [3]:

!pip install pyspark


In [4]:

!pip install findspark


### Instanciamos las librerías

In [11]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, min, max, avg, count
import findspark
findspark.init()

import pandas as pd
import matplotlib.pyplot as plt

## 2. Carga del conjunto de datos:

### Cargamos el set de datos "HeartDiseasePrediction" del repositorio de Kaggle

In [5]:
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


In [6]:
# descomprimimos el archivo
# !unzip heart_2020_cleaned.csv.zip
# !unzip heart_2022_with_nans.csv.zip
# mv nMiArchivo.zip archive.zip
!unzip archive.zip

Archive:  archive.zip
  inflating: 2020/heart_2020_cleaned.csv  
  inflating: 2022/heart_2022_no_nans.csv  
  inflating: 2022/heart_2022_with_nans.csv  
  inflating: vars_list_with_descriptions.txt  


In [9]:
! ls -l -h 2022/

total 212M
-rw-r--r-- 1 root root  79M Oct 12  2023 heart_2022_no_nans.csv
-rw-r--r-- 1 root root 134M Oct 12  2023 heart_2022_with_nans.csv


Iniciamos una sesión de Spark y cargamos el dataset

In [12]:
# Iniciar SparkSession

spark = SparkSession.builder.appName("HeartDiseasePrediction").getOrCreate()
# Cargar datos CSV
data = spark.read.csv("/content/2022/heart_2022_with_nans.csv", header=True)


### Verificamos el dataframe

In [13]:
# Imprimimos el
data.printSchema()

root
 |-- State: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- GeneralHealth: string (nullable = true)
 |-- PhysicalHealthDays: string (nullable = true)
 |-- MentalHealthDays: string (nullable = true)
 |-- LastCheckupTime: string (nullable = true)
 |-- PhysicalActivities: string (nullable = true)
 |-- SleepHours: string (nullable = true)
 |-- RemovedTeeth: string (nullable = true)
 |-- HadHeartAttack: string (nullable = true)
 |-- HadAngina: string (nullable = true)
 |-- HadStroke: string (nullable = true)
 |-- HadAsthma: string (nullable = true)
 |-- HadSkinCancer: string (nullable = true)
 |-- HadCOPD: string (nullable = true)
 |-- HadDepressiveDisorder: string (nullable = true)
 |-- HadKidneyDisease: string (nullable = true)
 |-- HadArthritis: string (nullable = true)
 |-- HadDiabetes: string (nullable = true)
 |-- DeafOrHardOfHearing: string (nullable = true)
 |-- BlindOrVisionDifficulty: string (nullable = true)
 |-- DifficultyConcentrating: string (nullable = t

In [14]:
# revisamos la cantidad de registro de nuestro dataset
row_number = data.count()
print('la cantidad de files es',row_number)

la cantidad de files es 445132


In [15]:
# revisamos la cantidad de columnas de nuestro dataset
column_number = len(data.dtypes)
print('la cantidad de columnas es',column_number)

la cantidad de columnas es 40


### Obtenemos un resumen del DataFrame

In [16]:
# Obtenemos un resumen estadístico de cada uno de los campos
data.describe().show()


+-------+-------+------+-------------+------------------+-----------------+--------------------+------------------+-----------------+------------+--------------+---------+---------+---------+-------------+-------+---------------------+----------------+------------+--------------------+-------------------+-----------------------+-----------------------+-----------------+-------------------------+-----------------+--------------------+--------------------+---------+---------------------+---------------+-------------------+------------------+------------------+---------------+----------+------------+-------------+--------------------+----------------+--------+
|summary|  State|   Sex|GeneralHealth|PhysicalHealthDays| MentalHealthDays|     LastCheckupTime|PhysicalActivities|       SleepHours|RemovedTeeth|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadCOPD|HadDepressiveDisorder|HadKidneyDisease|HadArthritis|         HadDiabetes|DeafOrHardOfHearing|BlindOrVisionDifficulty|Diff

In [17]:
data.columns

['State',
 'Sex',
 'GeneralHealth',
 'PhysicalHealthDays',
 'MentalHealthDays',
 'LastCheckupTime',
 'PhysicalActivities',
 'SleepHours',
 'RemovedTeeth',
 'HadHeartAttack',
 'HadAngina',
 'HadStroke',
 'HadAsthma',
 'HadSkinCancer',
 'HadCOPD',
 'HadDepressiveDisorder',
 'HadKidneyDisease',
 'HadArthritis',
 'HadDiabetes',
 'DeafOrHardOfHearing',
 'BlindOrVisionDifficulty',
 'DifficultyConcentrating',
 'DifficultyWalking',
 'DifficultyDressingBathing',
 'DifficultyErrands',
 'SmokerStatus',
 'ECigaretteUsage',
 'ChestScan',
 'RaceEthnicityCategory',
 'AgeCategory',
 'HeightInMeters',
 'WeightInKilograms',
 'BMI',
 'AlcoholDrinkers',
 'HIVTesting',
 'FluVaxLast12',
 'PneumoVaxEver',
 'TetanusLast10Tdap',
 'HighRiskLastYear',
 'CovidPos']

Revisamos detalladamente los campos del DataSet mediante un archivos de texto que contiene las descripciones

In [18]:
VAR_LIST_PATH = '/content/vars_list_with_descriptions.txt'
var_list_df = pd.read_csv(VAR_LIST_PATH, sep='|', header=None, names=['Variable', 'Description'])
var_list_df


,Variable,Description
0,_STATE,State FIPS Code
1,SEXVAR,Sex of Respondent
2,GENHLTH,Would you say that in general your health is:
3,PHYSHLTH,"Now thinking about your physical health, which..."
4,MENTHLTH,"Now thinking about your mental health, which i..."
5,CHECKUP1,About how long has it been since you last visi...
6,EXERANY2,"During the past month, other than your regular..."
7,SLEPTIM1,"On average, how many hours of sleep do you get..."
8,RMVTETH4,Not including teeth lost for injury or orthodo...
9,CVDINFR4,"(Ever told) you had a heart attack, also calle..."


## 3. Exploración de datos:

###Selección de variables
Seleccionamos las variables relevantes para el análisis. Por ejemplo, podemos seleccionar las variables relacionadas con los factores de riesgo de enfermedades cardíacas o simplemente una distribución por sexo


In [19]:

# probamos con la ejecuación de sentencias SQL

data.createOrReplaceTempView("HeartDisease") #temporary view
cardio_risk_factors  = spark.sql("SELECT * from HeartDisease")
cardio_risk_factors.printSchema()
cardio_risk_factors.show()


root
 |-- State: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- GeneralHealth: string (nullable = true)
 |-- PhysicalHealthDays: string (nullable = true)
 |-- MentalHealthDays: string (nullable = true)
 |-- LastCheckupTime: string (nullable = true)
 |-- PhysicalActivities: string (nullable = true)
 |-- SleepHours: string (nullable = true)
 |-- RemovedTeeth: string (nullable = true)
 |-- HadHeartAttack: string (nullable = true)
 |-- HadAngina: string (nullable = true)
 |-- HadStroke: string (nullable = true)
 |-- HadAsthma: string (nullable = true)
 |-- HadSkinCancer: string (nullable = true)
 |-- HadCOPD: string (nullable = true)
 |-- HadDepressiveDisorder: string (nullable = true)
 |-- HadKidneyDisease: string (nullable = true)
 |-- HadArthritis: string (nullable = true)
 |-- HadDiabetes: string (nullable = true)
 |-- DeafOrHardOfHearing: string (nullable = true)
 |-- BlindOrVisionDifficulty: string (nullable = true)
 |-- DifficultyConcentrating: string (nullable = t

In [20]:
groupDF = spark.sql('SELECT Sex, count(*) from HeartDisease group by Sex')
groupDF.show()

+------+--------+
|   Sex|count(1)|
+------+--------+
|Female|  235893|
|  Male|  209239|
+------+--------+



### Exploramos las distribuciones de las variables:

In [21]:
# data.hist(bins=10)
# plt.show()
data.select('AgeCategory').distinct().show()

# data.scatter(x='AgeCategory', y='Sex')
# plt.show()


+---------------+
|    AgeCategory|
+---------------+
|   Age 45 to 49|
|   Age 25 to 29|
|   Age 70 to 74|
|   Age 55 to 59|
|   Age 18 to 24|
|   Age 60 to 64|
|   Age 50 to 54|
|   Age 35 to 39|
|   Age 30 to 34|
|Age 80 or older|
|   Age 40 to 44|
|   Age 75 to 79|
|   Age 65 to 69|
|           NULL|
+---------------+



In [22]:
groupDF = spark.sql('SELECT AgeCategory, count(*) from HeartDisease group by AgeCategory')
groupDF.show()

+---------------+--------+
|    AgeCategory|count(1)|
+---------------+--------+
|   Age 45 to 49|   28531|
|   Age 25 to 29|   21990|
|           NULL|    9079|
|   Age 70 to 74|   43472|
|   Age 55 to 59|   36821|
|   Age 18 to 24|   26941|
|   Age 60 to 64|   44511|
|   Age 50 to 54|   33644|
|   Age 35 to 39|   28526|
|   Age 30 to 34|   25807|
|Age 80 or older|   36251|
|   Age 40 to 44|   29942|
|   Age 75 to 79|   32518|
|   Age 65 to 69|   47099|
+---------------+--------+



In [23]:
# revisamos nuevamente el resultado ordenado descendentemente
groupDF = spark.sql('SELECT AgeCategory, count(*) from HeartDisease group by AgeCategory order by count(1) desc ')
groupDF.show()

+---------------+--------+
|    AgeCategory|count(1)|
+---------------+--------+
|   Age 65 to 69|   47099|
|   Age 60 to 64|   44511|
|   Age 70 to 74|   43472|
|   Age 55 to 59|   36821|
|Age 80 or older|   36251|
|   Age 50 to 54|   33644|
|   Age 75 to 79|   32518|
|   Age 40 to 44|   29942|
|   Age 45 to 49|   28531|
|   Age 35 to 39|   28526|
|   Age 18 to 24|   26941|
|   Age 30 to 34|   25807|
|   Age 25 to 29|   21990|
|           NULL|    9079|
+---------------+--------+



In [30]:
# distribución por estado de salud (GeneralHealth)
groupDFGH = spark.sql('SELECT GeneralHealth, count(*) from HeartDisease group by GeneralHealth order by count(1) desc ')
groupDFGH.show()


+-------------+--------+
|GeneralHealth|count(1)|
+-------------+--------+
|    Very good|  148444|
|         Good|  143598|
|    Excellent|   71878|
|         Fair|   60273|
|         Poor|   19741|
|         NULL|    1198|
+-------------+--------+



In [44]:
# ss = groupDFGH['GeneralHealth'].unique()
# ss
# groupDFGH.filter(groupDFGH['GeneralHealth'] == 'Poor').show()
# groupDFGH.sort('GeneralHealth', ascending=False).show()
groupDFGH.sort('count(1)', ascending=False).show()



+-------------+--------+
|GeneralHealth|count(1)|
+-------------+--------+
|    Very good|  148444|
|         Good|  143598|
|    Excellent|   71878|
|         Fair|   60273|
|         Poor|   19741|
|         NULL|    1198|
+-------------+--------+



In [43]:
groupDFHH = spark.sql('SELECT HadHeartAttack,Sex, count(*) from HeartDisease group by HadHeartAttack,Sex order by count(1) desc ')
groupDFHH.show()

+--------------+------+--------+
|HadHeartAttack|   Sex|count(1)|
+--------------+------+--------+
|            No|Female|  224828|
|            No|  Male|  192131|
|           Yes|  Male|   15417|
|           Yes|Female|    9691|
|          NULL|  Male|    1691|
|          NULL|Female|    1374|
+--------------+------+--------+



In [45]:
groupDFHH.sort('Sex', ascending=True).show()

+--------------+------+--------+
|HadHeartAttack|   Sex|count(1)|
+--------------+------+--------+
|            No|Female|  224828|
|          NULL|Female|    1374|
|           Yes|Female|    9691|
|            No|  Male|  192131|
|           Yes|  Male|   15417|
|          NULL|  Male|    1691|
+--------------+------+--------+



Graficamos los resultados

In [46]:
# probamos la conversión de Dataframe PySpark a Pandas
from pyspark.sql import functions as F
dfP = data.groupBy('Sex').agg(F.count('AgeCategory'))

In [ ]:
# ahora procesamos la información en un dataFrame de Pandas
dfP.toPandas()

In [47]:
# Visualizamos el resultado con un gráfico
dataG = dfP.toPandas()

import plotly.express as px

fig = px.histogram(dataG, x="Sex",y="count(AgeCategory)")
fig.show()

# test.scatter(x='Sex', y='count(AgeCategory)')
# plt.show()

In [48]:
dfp = groupDF.toPandas()
fig = px.histogram(dfp, x="AgeCategory",y="count(1)")
fig.show()

In [49]:
#Graficamos el estado de salud

dfpGH = groupDFGH.toPandas()
fig = px.histogram(dfpGH, x="GeneralHealth",y="count(1)")
fig.show()

In [50]:
AgeC = spark.sql('SELECT AgeCategory, count(*) from HeartDisease group by AgeCategory')
AgeC = AgeC.toPandas()
AgeC.columns = ['RangoEdad','Cantidad']
AgeC.dtypes
# AgeC

fig = px.histogram(AgeC, x="RangoEdad", y="Cantidad")
fig.show()

In [52]:
dfHH

,HadHeartAttack,Sex,count(1)
0,No,Female,224828
1,No,Male,192131
2,Yes,Male,15417
3,Yes,Female,9691
4,None,Male,1691
5,None,Female,1374


In [54]:
dfHH =groupDFHH.toPandas()


fig = px.histogram(dfHH, x="HadHeartAttack", y="count(1)",color='Sex')
fig.show()


In [55]:
groupSSleep = spark.sql('SELECT Sex, BMI,HeightInMeters,SleepHours from HeartDisease where Sex="Male"')
groupSSleep.show()
SexSleep = groupSSleep.toPandas()

+----+-----+--------------+----------+
| Sex|  BMI|HeightInMeters|SleepHours|
+----+-----+--------------+----------+
|Male|26.08|           1.8|       7.0|
|Male|28.59|          1.73|       6.0|
|Male|32.78|           1.8|       8.0|
|Male| 26.5|           1.8|       9.0|
|Male|24.34|          1.93|       8.0|
|Male|24.39|          1.88|       4.0|
|Male|34.21|          1.73|       6.0|
|Male|19.13|          1.85|       8.0|
|Male|33.96|          1.75|      10.0|
|Male| NULL|          NULL|       6.0|
|Male|19.53|           1.8|       4.0|
|Male| 26.5|           1.8|       6.0|
|Male|29.84|          1.83|       6.0|
|Male|32.28|          1.68|       6.0|
|Male|41.35|          1.75|       8.0|
|Male|30.82|           1.6|       6.0|
|Male| NULL|          NULL|       8.0|
|Male|28.13|          1.73|       9.0|
|Male|26.54|          1.78|       8.0|
|Male|21.87|          1.91|       8.0|
+----+-----+--------------+----------+
only showing top 20 rows



In [56]:
fig = px.scatter(SexSleep, x="SleepHours", y="HeightInMeters",)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Conteno y eliminación de datos vacios o nulos

data.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in data.columns]).show()

## 4. Cálculos estadísticos:

Calcule medidas estadísticas básicas: Calcule medidas como la media, el mínimo, el máximo, el promedio y el recuento para cada variable:

In [57]:

stats = data.describe().toPandas()




In [ ]:
stats

Para revisar a mayor profundidad las diferenets funciones, ingresar al siguiente sitio:

https://spark.apache.org/docs/latest/programming-guide.html